In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/xlm-roberta-large-en-ru")
model = AutoModel.from_pretrained("DeepPavlov/xlm-roberta-large-en-ru")

c:\Users\mrars\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\mrars\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\mrars\.cache\huggingface\hub\models--DeepPavlov--xlm-roberta-large-en-ru. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Py

KeyboardInterrupt: 

In [170]:
import numpy as np
print(np.__version__)


2.3.1


In [137]:
import torch
from torch import nn
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from torch.optim import AdamW

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-uncased")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of the model checkpoint at google-bert/bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [70]:
model = AutoModel.from_pretrained("google-bert/bert-base-multilingual-uncased")

In [193]:
data_log = ['80.97.168.58 - gorshkovnikon [Тимофеев Серафим Антонович] (email: viktor15@example.net, pass: _7Ys%IVQp2) phone=8 826 587 31 88 addr="к. Армавир, наб. Грибоедова, д. 9/3 стр. 5/3, 937143" org=Медведева и партнеры ipv6=64be:ec33:33f1:1207:5d95:7f1:8710:a5','A support ticket was created by user jeffrey97 from organization Hill-Hall. The ticket relates to an issue reported by (575)679-5357x44829. The affected client has IP address 9468:d568:5c7e:3ece:55c2:8542:2c96:8ca1 and client ID 79dbb385-04e2-43ca-a6ec-dbd0ca04b741.']
ner_data = [[
  {
    "label": "ip",
    "start": 1,
    "end": 13,
    "value": "80.97.168.58"
  },
  {
    "label": "login",
    "start": 16,
    "end": 29,
    "value": "gorshkovnikon"
  },
  {
    "label": "fio",
    "start": 31,
    "end": 57,
    "value": "Тимофеев Серафим Антонович"
  },
  {
    "label": "email",
    "start": 67,
    "end": 87,
    "value": "viktor15@example.net"
  },
  {
    "label": "password",
    "start": 95,
    "end": 105,
    "value": "_7Ys%IVQp2"
  },
  {
    "label": "phone",
    "start": 113,
    "end": 128,
    "value": "8 826 587 31 88"
  },
  {
    "label": "address",
    "start": 135,
    "end": 187,
    "value": "к. Армавир, наб. Грибоедова, д. 9/3 стр. 5/3, 937143"
  },
  {
    "label": "org",
    "start": 193,
    "end": 213,
  },
  {
    "label": "ipv6",
    "start": 219,
    "end": 256,
    "value": "64be:ec33:33f1:1207:5d95:7f1:8710:a5a"
  }
],[{'label': 'login', 'start': 38, 'end': 47, 'value': 'jeffrey97'},
 {'label': 'org', 'start': 66, 'end': 75, 'value': 'Hill-Hall'},
 {'label': 'phone', 'start': 120, 'end': 139, 'value': '(575)679-5357x44829'},
 {'label': 'ipv6',
  'start': 176,
  'end': 215,
  'value': '9468:d568:5c7e:3ece:55c2:8542:2c96:8ca1'}]]

In [194]:
unique_labels = ['address', 'email', 'fio', 'ip', 'ipv6', 'login', 'org', 'password', 'phone']

In [199]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/xlm-roberta-large-en-ru")

In [245]:
def prepared_data(data_log,ner_data,unique_labels):
    df = pd.DataFrame({
    "sentence": data_log,
    "ner": ner_data,

  })
  #Creat label dictionary
    unique_labels_ner = ["O"] + sorted([f"B-{e}" for e in unique_labels] + [f"I-{e}" for e in unique_labels])
    label2id = {label: idx for idx, label in enumerate(unique_labels_ner)}
    id2label = {v: k for k, v in label2id.items()}
  
  # Tokenize text
    tokenized = df["sentence"].apply(lambda x: tokenizer.encode_plus(
        x,
        add_special_tokens=True,  # Adds [CLS] and [SEP]
        max_length=256,           # Pad/truncate to max length
        padding='max_length',
        truncation=True,
        return_offsets_mapping=True,
        return_attention_mask=True,
        return_tensors='pt'       # Return PyTorch tensors
    ))
    df["input_ids"] = tokenized.apply(lambda x: x["input_ids"].squeeze(0))
    df["attention_mask"] = tokenized.apply(lambda x: x["attention_mask"].squeeze(0))
    df["offset_mapping"] = tokenized.apply(lambda x: x["offset_mapping"].squeeze(0))
    return df,label2id,id2label,unique_labels_ner
  #
df,label2id,id2label,unique_labels_ner = prepared_data(data_log,ner_data,unique_labels)

In [ ]:
def create_labels(ners,offsets,label2id):
# Assign labels to tokens
    total_labels = []
    for ner,offset in zip(ners,offsets):
        labels = [label2id["O"]] * len(offset)
        for entity in ner:
            ent_start, ent_end = entity["start"] - 1, entity["end"] -1
            ent_label = entity["label"]
            for idx,(start, end) in enumerate(offset):
                custom_start = start
                custom_end = end
                if (start,end) == (0,0):
                    labels[idx] = -100
                elif((custom_start + 1>= ent_start) and (custom_end <= ent_end)):
                    if (custom_start + 1)== ent_start:
                        labels[idx] = label2id[f"B-{ent_label}"]
                    else:
                        labels[idx] = label2id[f"I-{ent_label}"]

        total_labels.append(labels)
    return total_labels
df['labels'] = create_labels(df["ner"],df["offset_mapping"],label2id)

In [265]:
def mark_label(labels,idx,label2id,label_ner,start_ner =-1,start_token = 1):
  if (start_token < start_ner):
    labels[idx] = label2id[f"B-{label_ner}"]
  else:
    labels[idx] = label2id[f"I-{label_ner}"]
  return labels


In [246]:
df

,sentence,ner,input_ids,attention_mask,offset_mapping
0,80.97.168.58 - gorshkovnikon [Тимофеев Серафим...,"[{'label': 'ip', 'start': 1, 'end': 13, 'value...","[tensor(0), tensor(1608), tensor(5), tensor(57...","[tensor(1), tensor(1), tensor(1), tensor(1), t...","[[tensor(0), tensor(0)], [tensor(0), tensor(2)..."
1,A support ticket was created by user jeffrey97...,"[{'label': 'login', 'start': 38, 'end': 47, 'v...","[tensor(0), tensor(62), tensor(3798), tensor(2...","[tensor(1), tensor(1), tensor(1), tensor(1), t...","[[tensor(0), tensor(0)], [tensor(0), tensor(1)..."


In [267]:
def create_labels(ners,offsets,label2id):
  total_labels = []
  for ner,offset in zip(ners,offsets):
    labels = [label2id["O"]] * len(offset)
    idx = 0
    for item in offset:
      start_token = item[0]
      end_token = item[1]
      if start_token == 0 and end_token == 0:
        labels[idx] = -100 
      else:
        for entity in ner:
          start_ner = entity["start"]
          end_ner = entity["end"]
          label_ner = entity["label"]
          if end_token >= start_ner and end_token < end_ner:
            labels = mark_label(labels,idx,label2id,label_ner,start_ner,start_token)
            break
          elif end_ner > (start_token +1) and start_token > start_ner:
            labels = mark_label(labels,idx,label2id,label_ner)
            break
      idx +=1
    total_labels.append(labels)
  return total_labels
df['labels'] = create_labels(df["ner"],df["offset_mapping"],label2id)






    


In [261]:
df['sentence'][1]

'A support ticket was created by user jeffrey97 from organization Hill-Hall. The ticket relates to an issue reported by (575)679-5357x44829. The affected client has IP address 9468:d568:5c7e:3ece:55c2:8542:2c96:8ca1 and client ID 79dbb385-04e2-43ca-a6ec-dbd0ca04b741.'

In [262]:
df['ner'][1]

[{'label': 'login', 'start': 38, 'end': 47, 'value': 'jeffrey97'},
 {'label': 'org', 'start': 66, 'end': 75, 'value': 'Hill-Hall'},
 {'label': 'phone', 'start': 120, 'end': 139, 'value': '(575)679-5357x44829'},
 {'label': 'ipv6',
  'start': 176,
  'end': 215,
  'value': '9468:d568:5c7e:3ece:55c2:8542:2c96:8ca1'}]

In [268]:
def test_tokens(labels,input_ids,offset):
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    token_labels =  [id2label[l] if l != -100 else "IGNORED" for l in labels]
    for t, l, o in zip(tokens, token_labels,offset):
        print(f"{t:15} → {l}, {o}")
test_tokens(df['labels'][1],df['input_ids'][1],df["offset_mapping"][1])

<s>             → IGNORED, tensor([0, 0])
▁A              → O, tensor([0, 1])
▁support        → O, tensor([1, 9])
▁ticket         → O, tensor([ 9, 16])
▁was            → O, tensor([16, 20])
▁created        → O, tensor([20, 28])
▁by             → O, tensor([28, 31])
▁user           → O, tensor([31, 36])
▁je             → B-login, tensor([36, 39])
ff              → I-login, tensor([39, 41])
rey             → I-login, tensor([41, 44])
97              → I-login, tensor([44, 46])
▁from           → O, tensor([46, 51])
▁organization   → O, tensor([51, 64])
▁Hill           → B-org, tensor([64, 69])
-               → I-org, tensor([69, 70])
H               → I-org, tensor([70, 71])
all             → I-org, tensor([71, 74])
.               → O, tensor([74, 75])
▁The            → O, tensor([75, 79])
▁ticket         → O, tensor([79, 86])
▁relat          → O, tensor([86, 92])
es              → O, tensor([92, 94])
▁to             → O, tensor([94, 97])
▁an             → O, tensor([ 97, 100])
▁issue  

In [176]:
data_log2 = ['A support ticket was created by user jeffrey97 from organization Hill-Hall. The ticket relates to an issue reported by (575)679-5357x44829. The affected client has IP address 9468:d568:5c7e:3ece:55c2:8542:2c96:8ca1 and client ID 79dbb385-04e2-43ca-a6ec-dbd0ca04b741.']
ner_data2 = [{'label': 'login', 'start': 38, 'end': 47, 'value': 'jeffrey97'},
 {'label': 'org', 'start': 66, 'end': 75, 'value': 'Hill-Hall'},
 {'label': 'phone', 'start': 120, 'end': 139, 'value': '(575)679-5357x44829'},
 {'label': 'ipv6',
  'start': 176,
  'end': 215,
  'value': '9468:d568:5c7e:3ece:55c2:8542:2c96:8ca1'}]

In [174]:
unique_labels = ['address', 'email', 'fio', 'ip', 'ipv6', 'login', 'org', 'password', 'phone']

In [132]:
def prepared_data(data_log,ner_data,unique_labels):
  df = pd.DataFrame({
    "sentence": data_log,
    "ner": ner_data,

  })
  #Creat label dictionary
  unique_labels_ner = ["O"] + sorted([f"B-{e}" for e in unique_labels] + [f"I-{e}" for e in unique_labels])
  label2id = {label: idx for idx, label in enumerate(unique_labels_ner)}
  id2label = {v: k for k, v in label2id.items()}
  
  # Tokenize text
  tokenized = df["sentence"].apply(lambda x: tokenizer.encode_plus(
        x,
        add_special_tokens=True,  # Adds [CLS] and [SEP]
        max_length=128,           # Pad/truncate to max length
        padding='max_length',
        truncation=True,
        return_offsets_mapping=True,
        return_attention_mask=True,
        return_tensors='pt'       # Return PyTorch tensors
    ))
  df["input_ids"] = tokenized.apply(lambda x: x["input_ids"].squeeze(0))
  df["attention_mask"] = tokenized.apply(lambda x: x["attention_mask"].squeeze(0))
  df["offset_mapping"] = tokenized.apply(lambda x: x["offset_mapping"].squeeze(0))
  return df,label2id,id2label,unique_labels_ner
  #
df,label2id,id2label,unique_labels_ner = prepared_data(data_log,ner_data,unique_labels)

In [144]:
unique_labels_ner

['O',
 'B-Address',
 'B-FIO',
 'B-Organization',
 'B-Phone',
 'B-email',
 'B-ipv4',
 'B-ipv6',
 'B-login',
 'B-password',
 'I-Address',
 'I-FIO',
 'I-Organization',
 'I-Phone',
 'I-email',
 'I-ipv4',
 'I-ipv6',
 'I-login',
 'I-password']

In [133]:
def create_labels(ners,offsets,label2id):
# Assign labels to tokens
  total_labels = []
  for ner,offset in zip(ners,offsets):
    labels = [label2id["O"]] * len(offset)
    for entity in ner:
        ent_start, ent_end = entity["start"], entity["end"]
        ent_label = entity["label"]
        for idx, (start, end) in enumerate(offset[1:-1]):
          idx += 1
          start += 1
          if (start >= ent_start and end <= ent_end):
              if (start)== ent_start:
                  labels[idx] = label2id[f"B-{ent_label}"]
              else:
                  labels[idx] = label2id[f"I-{ent_label}"]
    total_labels.append(labels)
  return total_labels
df['labels'] = create_labels(df["ner"],df["offset_mapping"],label2id)

In [143]:
def test_tokens_to_list_file(labels, input_ids, tokenizer, id2label, output_file="ner_list.txt"):
    """
    Writes tokens and their labels as a Python list of lists to a text file.

    Args:
        labels (list of list): List of label IDs for each sentence.
        input_ids (list of list): List of token IDs for each sentence.
        tokenizer: Hugging Face tokenizer.
        id2label (dict): Mapping from label IDs to label names.
        output_file (str): Path to output file.
    """
    all_sentences = []

    for label_seq, input_id_seq in zip(labels, input_ids):
        tokens = tokenizer.convert_ids_to_tokens(input_id_seq)
        token_labels = [id2label[l] for l in label_seq]

        # Filter out special tokens
        filtered = [
            (token, tag)
            for token, tag in zip(tokens, token_labels)
            if token not in tokenizer.all_special_tokens
        ]
        all_sentences.append(filtered)

    # Write to file as Python list
    with open(output_file, "w", encoding="utf-8") as f:
        f.write("[\n")
        for sentence in all_sentences:
            f.write("  [\n")
            for token, tag in sentence:
                f.write(f"    ({repr(token)}, {repr(tag)}),\n")
            f.write("  ],\n")
        f.write("]\n")

    print(f"✅ Token-label pairs saved as Python list in {output_file}")

test_tokens_to_list_file(
    df['labels'],
    df['input_ids'],
    tokenizer,
    id2label,
    output_file="ner_output.txt"
)

✅ Token-label pairs saved as Python list in ner_output.txt


In [134]:
def test_tokens(labels,input_ids):
  for label,input_id in zip(labels,input_ids):
    tokens = tokenizer.convert_ids_to_tokens(input_id)
    token_labels = [id2label[l] for l in label]
    for t, l in zip(tokens, token_labels):
        print(f"{t:15} → {l}")
test_tokens(df['labels'],df['input_ids'])

[CLS]           → O
80              → B-ipv4
.               → I-ipv4
97              → I-ipv4
.               → I-ipv4
168             → I-ipv4
.               → I-ipv4
58              → I-ipv4
-               → O
go              → I-login
##rs            → I-login
##h             → I-login
##kov           → I-login
##nik           → I-login
##on            → I-login
[               → I-FIO
Тим             → I-FIO
##о             → I-FIO
##фе            → I-FIO
##ев            → I-FIO
Се              → I-FIO
##раф           → I-FIO
##им            → I-FIO
Антон           → I-FIO
##ович          → I-FIO
]               → I-FIO
(               → O
email           → O
:               → I-email
vik             → I-email
##tor           → I-email
##15            → I-email
@               → I-email
example         → I-email
.               → I-email
net             → I-email
,               → I-email
pass            → O
:               → I-password
_               → I-password
7            

In [135]:
class TokenClassifierHead(nn.Module):
    def __init__(self, hidden_size, num_labels):
        """
        Classification head for token-level predictions.
        """
        super(TokenClassifierHead, self).__init__()
        self.classifier = nn.Linear(hidden_size, num_labels)

    def forward(self, x):
        """
        Args:
            x: tensor of shape [batch_size, seq_len, hidden_size]
        Returns:
            logits: tensor of shape [batch_size, seq_len, num_labels]
        """
        logits = self.classifier(x)
        return logits
    

In [136]:
class BERTForNER(nn.Module):
    def __init__(self, model, num_labels):
        """
        BERT + token classification head for NER.
        """
        super(BERTForNER, self).__init__()
        self.bert = model
        hidden_size = self.bert.config.hidden_size
        self.classifier = TokenClassifierHead(hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None):
        """
        Args:
            input_ids: [batch_size, seq_len]
            attention_mask: [batch_size, seq_len]
            token_type_ids: [batch_size, seq_len]
        Returns:
            logits: [batch_size, seq_len, num_labels]
        """
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )

        # Get all token embeddings
        sequence_output = outputs.last_hidden_state  # [batch_size, seq_len, hidden_size]

        # Predict logits for each token
        logits = self.classifier(sequence_output)  # [batch_size, seq_len, num_labels]
        return logits


In [139]:
type(df['input_ids'][0])

torch.Tensor

In [152]:
from torch.utils.data import Dataset

class NERDataset(Dataset):
    def __init__(self, input_ids,attention_mask, labels):
        """
        Args:
            encodings: tokenizer output (input_ids, attention_mask, etc.)
            labels: list of BIO tag IDs aligned with tokens
        """
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item ={}
        item['input_ids'] =  torch.tensor(self.input_ids[idx])
        item['attention_mask']  = torch.tensor(self.attention_mask[idx])
        item["labels"] =  torch.tensor(self.labels[idx])
        return item

In [165]:
def train(model,df,unique_labels_ner,epoch):
    num_labels = len(unique_labels_ner)
    input_ids = df['input_ids']
    attention_mask = df['attention_mask']
    labels = df['labels']
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_test = BERTForNER(model, num_labels).to(device)
    optimizer = AdamW(model_test.parameters(), lr=5e-5)
    criterion = nn.CrossEntropyLoss(ignore_index=-100)
    dataset = NERDataset(input_ids,attention_mask, labels)  # Assume you preprocessed your data
    dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

    epochs = epoch
    for epoch in range(epochs):
        model_test.train()
        total_loss = 0

        for batch in dataloader:
            optimizer.zero_grad()
            print(type(batch["input_ids"]))
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            if len(input_ids.shape) == 1:
                input_ids = input_ids.unsqueeze(0)
                attention_mask = attention_mask.unsqueeze(0)
            # Forward
            logits = model_test(input_ids, attention_mask)

            # Reshape logits and labels for loss
            loss = criterion(
                logits.view(-1, num_labels),  # [batch * seq_len, num_labels]
                labels.view(-1)               # [batch * seq_len]
            )
            total_loss += loss.item()

            # Backward
            loss.backward()
            optimizer.step()

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{epochs} - Loss: {avg_loss:.4f}")
    return model_test

In [166]:
model_test = train(model,df,unique_labels_ner,5)

C:\Users\mrars\AppData\Local\Temp\ipykernel_32884\2707286675.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['input_ids'] =  torch.tensor(self.input_ids[idx])
C:\Users\mrars\AppData\Local\Temp\ipykernel_32884\2707286675.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['attention_mask']  = torch.tensor(self.attention_mask[idx])


<class 'torch.Tensor'>
Epoch 1/5 - Loss: 3.2802
<class 'torch.Tensor'>
Epoch 2/5 - Loss: 1.9860
<class 'torch.Tensor'>
Epoch 3/5 - Loss: 1.2943
<class 'torch.Tensor'>
Epoch 4/5 - Loss: 0.9800
<class 'torch.Tensor'>
Epoch 5/5 - Loss: 0.8118


In [164]:
type(model_test)

NoneType

In [167]:
def predict(text, model, tokenizer, id2label, device="cpu"):
    """
    Run NER prediction on raw text.

    Args:
        text (str): Raw input text.
        model (nn.Module): Your trained BERTForNER model.
        tokenizer: Hugging Face tokenizer.
        id2label (dict): Mapping from label IDs to label names.
        device (str): "cpu" or "cuda".
    Returns:
        List of (token, predicted_label)
    """
    # Tokenize input text
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,  # Adds [CLS] and [SEP]
        max_length=128,           # Pad/truncate to max length
        padding='max_length',
        truncation=True,
        return_offsets_mapping=True,
        return_attention_mask=True,
        return_tensors='pt'       # Return PyTorch tensors
    )
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    # Set model to eval mode
    model.eval()
    with torch.no_grad():
        # Forward pass
        logits = model(input_ids, attention_mask)
        predictions = torch.argmax(logits, dim=-1)  # [batch_size, seq_len]

    # Convert IDs to tokens and labels
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])
    predicted_labels = [id2label[p.item()] for p in predictions[0]]

    # Filter out special tokens ([CLS], [SEP], [PAD])
    filtered_results = []
    for token, label in zip(tokens, predicted_labels):
        if token not in tokenizer.all_special_tokens:
            filtered_results.append((token, label))

    return filtered_results


In [169]:
# Assume your trained model + tokenizer are loaded
text = data_log[0]

results = predict(text, model_test, tokenizer, id2label, device="cuda")
for token, label in results:
    print(f"{token:15} → {label}")


80              → I-FIO
.               → I-FIO
97              → I-ipv4
.               → I-FIO
168             → I-ipv4
.               → I-FIO
58              → I-ipv4
-               → I-FIO
go              → I-FIO
##rs            → I-FIO
##h             → I-FIO
##kov           → I-FIO
##nik           → I-FIO
##on            → I-FIO
[               → I-FIO
Тим             → I-FIO
##о             → I-FIO
##фе            → I-FIO
##ев            → I-FIO
Се              → I-FIO
##раф           → I-FIO
##им            → I-FIO
Антон           → I-FIO
##ович          → I-FIO
]               → I-FIO
(               → O
email           → O
:               → I-password
vik             → I-password
##tor           → I-password
##15            → I-password
@               → I-email
example         → I-password
.               → I-email
net             → I-password
,               → I-password
pass            → I-password
:               → I-password
_               → I-password
7              